In [34]:
# Imports

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import tensorflow as tf
import numpy as np
import pandas as pd
from itertools import *
import sklearn
import math
import random
import sys
import multiprocessing
import scipy
from joblib import Parallel, delayed
import threading
nproc = max(1, multiprocessing.cpu_count() - 1)

if 'utils' not in sys.path:
    sys.path.append('utils')

import data_loader

# Warnings

import warnings
warnings.filterwarnings('ignore')

# Idempotent, cached data retrieval script

print(data_loader.load_chromosome.__doc__)
train_df, test_df, train_ix, test_ix, train_tissues, tfs = \
    data_loader.load_chromosome_cached('1')
    
sess = tf.InteractiveSession()

Idempotent data loading. For a given chromosome n (a string).
    
    Returns (train_df, test_df, train_ix, test_ix, train_tissues, tfs)
    
    The first two are the train and test dataframes, and test_ix are the
    values in test_df['assayed'] that are missing and need to be imputed (with the
    correct answer being in test_df['filled'] in the corresponding locations.
    train_ix are the assayed (known) methylation values from limited microarray
    sampling (e.g., test_df['assayed'].iloc[train_ix] can be used for prediction of
    test_df['filled'].iloc[test_ix], and the former should be about equal to
    test_df['filled'].iloc[train_ix] (two different ways of sampling methylation).
    
    Imports genetic context and adds those columns to the parameter df, returning
    a merged one. tfs is the list of names of new transcription
    factors.
    
    train_tissues is a list of the names of columns with chromosome methylation values.
    
    Note that loading from scratch ma

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f019cffff10>> ignored


In [2]:
# Perhaps there are obvious sequence trends?
def local_impute(data):
    #http://stackoverflow.com/questions/9537543/replace-nans-in-numpy-array-with-closest-non-nan-value
    mask = np.isnan(data)
    data[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data[~mask])
    return data

# Do mean imputation on our training data.
def mean_impute(data):
    mask = np.isnan(data)
    data[mask] = float(data.mean()) # just = m messes with serialization
    return data
train_df_imp = train_df
train_df_int = train_df
for i in train_tissues:
    train_df_imp[i] = mean_impute(train_df[i].copy())
    train_df_int[i] = local_impute(train_df[i].copy())
print('nans in mean-imputed', np.isnan(train_df_imp[train_tissues]).sum().sum())
print('nans in interpolated', np.isnan(train_df_int[train_tissues]).sum().sum())

nans in mean-imputed 0
nans in interpolated 0


In [3]:
# Copied pretty much directly from sklearn

# Returns a TensorFlow scalar with the size of the i-th dimension for
# the parameter tensor x.
def tf_get_shape(x, i):
    return tf.squeeze(tf.slice(tf.shape(x), [i], [1])) 

def tf_nrows(x):
    return tf_get_shape(x, 0)

def tf_ncols(x):
    return tf_get_shape(x, 1)

# Simultaneous K-cluster likelihood computation.
# X is NxD, mus is KxD, sigmas is KxD
# Output is KxN likelihoods for each sample in each cluster.
def tf_log_normals(X, mus, sigmas):
    # p(X) = sqrt(a * b * c)
    # a = (2 pi)^(-p)
    # b = det(sigma)^(-1)
    # c = exp(-(x - mu)^T sigma^(-1) (x - mu)) [expanded for numerical stability]
    #
    # Below we make simplifications since sigma is diag
    
    D = tf_ncols(mus)
    XT = tf.transpose(X) # pxN
    invsig = tf.inv(sigmas)
    
    loga = -tf.cast(D, 'float64') * tf.log(tf.constant(2 * np.pi, dtype='float64')) # scalar
    logb = tf.reduce_sum(tf.log(invsig), 1, keep_dims=True) # Kx1
    logc =  \
        - tf.reduce_sum(invsig * tf.square(mus), 1, keep_dims=True) \
        + 2 * tf.matmul(invsig * mus, XT) \
        - tf.matmul(invsig, tf.square(XT)) # KxN
    
    return 0.5 * (loga + logb + logc)

# Stably log-sum-exps likelihood along rows.
# Reduces KxN tensor L to 1xN tensor
def tf_log_sum_exp(L):
    maxs = tf.reduce_max(L, 0, keep_dims=True) # 1xN
    return tf.log(tf.reduce_sum(tf.exp(L - maxs), 0, keep_dims=True)) + maxs

# X is NxD, mus is KxD, sigmas KxD, alphas is K
# output is KxN log likelihoods.
def tf_log_likelihood(X, mus, sigmas, alphas):
    alphas = tf.expand_dims(alphas, 1) # Kx1
    return tf_log_normals(X, mus, sigmas) + tf.log(alphas) # KxN

# X is NxD, mus is KxD, sigmas KxD, alphas is K
# output is 1xN log probability for each sample, KxN responsibilities
def estep(X, mus, sigmas, alphas):
    log_likelihoods = tf_log_likelihood(X, mus, sigmas, alphas)
    sample_log_prob = tf_log_sum_exp(log_likelihoods) # 1xN
    return sample_log_prob, tf.exp(log_likelihoods - sample_log_prob)

EPS = np.finfo(float).eps
MIN_COVAR = 0.1
# X is NxD, resp is KxN (and normalized along axis 0)
# Returns maximize step means, covariance, and cluster priors,
# which have dimension KxD, KxD, and K, respectively
def mstep(X, resp):
    weights = tf.reduce_sum(resp, 1) # K
    invweights = tf.expand_dims(tf.inv(weights + 10 * EPS), 1) # Kx1
    alphas = EPS + weights / (tf.reduce_sum(weights) + 10 * EPS) # K
    weighted_cluster_sum = tf.matmul(resp, X) # KxD 
    mus = weighted_cluster_sum * invweights
    avg_X2 = tf.matmul(resp, tf.square(X)) * invweights
    avg_mu2 = tf.square(mus)
    avg_X_mu = mus * weighted_cluster_sum * invweights
    sigmas = avg_X2 - 2 * avg_X_mu + avg_mu2 + MIN_COVAR
    # (x - mu) (x-mu)^T for banded. 
    return mus, sigmas, alphas

In [4]:
# Make sure N, D are small so the numerically unstable verification code
# doesn't underflow.
N = 5
D = 10

X = np.random.normal(size=(N, D))
mu = X.mean(axis=0)
sigma = X.std(axis=0)
mus = np.array([mu, mu, mu * 2])
sigmas = np.array([sigma, sigma * 2, sigma])
K = len(sigmas)
alphas = np.random.dirichlet(np.ones(K), 1)[0]

mean_ll, resp = sess.run(estep(*(tf.constant(x) for x in (X, mus, sigmas, alphas))))
def normal_likelihoods(X, mu, sigma):
    exponent = -np.dot((X - mu[np.newaxis, :]) ** 2, 1 / sigma) / 2
    return (2 * np.pi) ** (-D / 2) * np.prod(sigma) ** (-1 / 2) * np.exp(exponent)
actual = np.array([normal_likelihoods(X, mu, sigma) for mu, sigma in zip(mus, sigmas)])
actual = sklearn.preprocessing.normalize(actual * alphas[:, np.newaxis], norm='l1', axis=0)
resp = sklearn.preprocessing.normalize(resp, norm='l1', axis=0)
print('actual likelihoods', actual)
print('log likelihoods   ', resp)
rmses = np.sqrt(sklearn.metrics.mean_squared_error(actual.T, resp.T, multioutput='raw_values'))
for i, rmse in enumerate(rmses):
    print('K={} rmse={}'.format(i, rmse))

actual likelihoods [[ 0.55608512  0.83276787  0.3352924   0.53247019  0.8850091 ]
 [ 0.00546339  0.02183937  0.00645537  0.00612281  0.01459697]
 [ 0.43845149  0.14539276  0.65825223  0.461407    0.10039392]]
log likelihoods    [[ 0.55608512  0.83276787  0.3352924   0.53247019  0.8850091 ]
 [ 0.00546339  0.02183937  0.00645537  0.00612281  0.01459697]
 [ 0.43845149  0.14539276  0.65825223  0.461407    0.10039392]]
K=0 rmse=2.9373740229761033e-16
K=1 rmse=2.854661289398182e-17
K=2 rmse=2.4763200404893063e-16


In [5]:
# Similar pattern to
# https://gist.github.com/narphorium/d06b7ed234287e319f18

#todo try initializing covar to emprical cv computed from kmeans labels

# Runs up to max_steps EM iterations, stopping earlier if log likelihood improves
# less than tol.
# X should be an NxD data matrix, initial_mus should be KxD
# max_steps should be an int, tol should be a float.
def fit_em(X, initial_mus, max_steps, tol, sess):
    N, D = X.shape
    K, Dmu = initial_mus.shape
    assert D == Dmu
        
    mus0 = initial_mus
    sigmas0 = np.tile(np.var(X, axis=0), (K, 1))
    alphas0 = np.ones(K) / K
    X = tf.constant(X)
    
    mus, sigmas, alphas = (tf.Variable(x, dtype='float64') for x in [mus0, sigmas0, alphas0])
    
    sess.run(tf.initialize_all_variables())
    all_ll, resp = estep(X, mus, sigmas, alphas)
    cmus, csigmas, calphas = mstep(X, resp)
    update_mus_step = tf.assign(mus, cmus)
    update_sigmas_step = tf.assign(sigmas, csigmas)
    update_alphas_step = tf.assign(alphas, calphas)     
    
    init_op = tf.initialize_all_variables()
    ll = prev_ll = -np.inf
                         
    with tf.Session() as sess2:
        sess2.run(init_op)
        for i in range(max_steps):
            ll = sess2.run(tf.reduce_mean(all_ll))
            sess2.run((update_mus_step, update_sigmas_step, update_alphas_step))
            print('EM iteration', i, 'log likelihood', ll)
            if abs(ll - prev_ll) < tol:
                break
            prev_ll = ll
        m, s, a = sess2.run((mus, sigmas, alphas))
    
    return ll, m, s, a

In [6]:
# Given a set of partial observations xs each of dimension O < D for a fitted GMM model with 
# K cluster priors alpha, KxD means mus, and KxD diagonal covariances sigmas,
# returns the weighted sum of normals for the remaining D - O dimensions.
#
# Returns posterior_mus, posterior_sigmas, posterior_prior,
# of dimensions:
# Kx(D-O), Kx(D-O), NxK, respectively (each mu, sigma is the same for all posteriors).
# NxK, NxKxD, NxKxD, respectively, for each x in xs, total of N.
def marginal_posterior(xs, mus, sigmas, alphas, sess):
    # https://gbhqed.wordpress.com/2010/02/21/conditional-and-marginal-distributions-of-a-multivariate-gaussian/
    # diagonal case is easy:
    # https://en.wikipedia.org/wiki/Schur_complement#Applications_to_probability_theory_and_statistics
    O = xs.shape[1]
    D = mus.shape[1]
    observed_mus, observed_sigmas = (tf.constant(a, dtype='float64')
                                     for a in (mus[:,0:O], sigmas[:, 0:O]))
    ll = tf_log_likelihood(xs, observed_mus, observed_sigmas, alphas) # KxN
    norm = tf_log_sum_exp(ll) # 1xN
    ll, norm = sess.run((ll, norm))
    return mus[:, O:D], sigmas[:, O:D], np.transpose(ll / norm)

# A "sparser" estimate which just uses the most likely cluster's mean as the estimate.
def argmax_exp(mus, sigmas, alphas):
    i = np.argmax(alphas)
    return mus[i]

In [7]:
n_samples = 25
np.random.seed(0)

def MVN(shear, shift):
    rs = np.random.randn(n_samples, 2)
    return np.dot(rs, shear.T) + shift
G1 = MVN(np.identity(2), np.array([20, 20]))
G2 = MVN(np.array([[0., 3.5], [-0.7, .7]]), np.zeros(2))
G3 = MVN(np.array([[-1, 0.8], [0, 4]]), np.array([10, 8]))

# concatenate the two datasets into the final training set
X = np.vstack([G1, G2, G3])
rx = np.random.choice(range(len(X)), 3, replace=False)

# http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html
_, m, s, a = fit_em(X, X[rx], 100, EPS)
x = np.linspace(-20.0, 30.0)
y = np.linspace(-20.0, 40.0)
xx, yy = np.meshgrid(x, y)
pts = np.array([xx.ravel(), yy.ravel()]).T
ll = -estep(pts, m, s, a)[0].eval()
print('means\n{}\ncov\n{}\ncluster priors\n{}'.format(m, s, a))
CS = plt.contour(xx, yy, ll.reshape(xx.shape), norm=LogNorm(vmin=1.0, vmax=1000.0),
                levels=np.logspace(0, 3, 20))
plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X[:, 0], X[:, 1])
plt.show()

TypeError: fit_em() missing 1 required positional argument: 'sess'

In [8]:
# Make this into a nice image.
ys = [-10, 0, 2.5, 10, 20]
pts = np.arange(-20, 30, 0.1, dtype='float64').reshape(-1, 1)
ys = np.array(ys, dtype='float64').reshape(-1, 1)
# Reverse m, s, a, since we know 'y'
mr, sr = (x[:, ::-1] for x in (m, s))
mm, sm, ams = marginal_posterior(ys, mr, sr, a)
for i, y in enumerate(ys.reshape(-1)):
    print('y =', y)
    ll = estep(pts, mm, sm, ams[i])[0].eval().reshape(-1)
    plt.plot(pts, np.exp(ll) / 10 + y)
    #plt.show()

ys = np.arange(-20, 30, 0.05, dtype='float64').reshape(-1, 1)
mm, sm, ams = marginal_posterior(ys, mr, sr, a)
xs = [argmax_exp(mm, sm, am) for am in ams]
plt.plot(xs, ys, label='argmax cluster mu')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

NameError: name 'm' is not defined

In [27]:
# N = 10
N = 33
p = 20 # D is better?
K = 6
k = 0 # maximum band offset (0 is just diagonal)
tissues_to_cluster = train_tissues
X_np = train_df_imp[train_tissues].values.transpose()

np.random.seed(2)
rc = np.random.choice(range(len(train_df)), D, replace=False)
rmu = np.random.choice(range(N), K, replace=False)

np.random.shuffle(X_np)
per = N // K
splits = [X_np[i:i+per] for i in range(0, N // K * K, per)]


# X_trunc = X_np[:N, rc]
X_trunc = X_np[:N]
mu_init = X_trunc[rmu]
# mu_init = np.array([x.mean(axis=0) for x in splits])
#mu_init = X_np[19:22]


print(mu_init.shape)

(6, 379551)


In [10]:
print(X_np.shape)



(34, 379551)


In [11]:
## Implement custom GMM
#todo check float32/float64 conversion for speedup
#todo scale (standardize) data before running
#todo kmeans warmup (just in numpy?)
#todo mincovar
lp, m, s, a = fit_em(X_trunc, mu_init, 100, EPS)
print('mean log likelihood\n{}\nmeans\n{}\ncov\n{}\ncluster priors\n{}'
      .format(lp, m, s, a))

TypeError: fit_em() missing 1 required positional argument: 'sess'

In [12]:
o = np.ones(len(train_df))
o[train_ix] = 0
o[test_ix] = 0
unobserved_untested_ix = np.where(o)[0]
o = np.zeros(len(train_df))
o[test_ix] = 1
unobserved_tested_ix = np.where(o)[0]

In [13]:
perm = np.hstack((train_ix, unobserved_tested_ix, unobserved_untested_ix))
print(len(train_ix))
print(len(unobserved_tested_ix))
print(len(unobserved_untested_ix))
print(perm)

7523
368411
3617
[  1084   1154   1214 ..., 378792 378806 379168]


In [14]:
observed = test_df['filled'][train_ix].values
print(observed.shape)
len(perm), len(train_df)

(7523,)


(379551, 379551)

In [15]:
import scipy
for t in train_tissues:
    print('tissue', t, 'correlations')
    for i, x in enumerate(m):
        print('    mean', i, scipy.stats.pearsonr(x, train_df[t])[0])

tissue b0 correlations


NameError: name 'm' is not defined

In [16]:
mm, ms = (x[:, perm] for x in (m, s))
print(len(mm[0]), len(ms[0]), len(a))
print(observed.reshape(1, len(observed)))
print(observed.shape)
mm, sm, ams = marginal_posterior(observed.reshape(1, len(observed)), mm, ms, a)
print(mm.shape, sm.shape, ams.shape)
pred = argmax_exp(mm, sm, ams[0])[:len(unobserved_tested_ix)]
print(pred)
actual = test_df['filled'][unobserved_tested_ix]
print(len(pred), len(actual))
# print('rmse', math.sqrt(actual.sub(pred).pow(2).mean())) # rmse of GMM
print('rmse', np.sqrt(sklearn.metrics.mean_squared_error(actual, pred)))
print('r2', sklearn.metrics.r2_score(actual, pred))

NameError: name 'm' is not defined

In [37]:
print(mm.shape)
print(ms.shape)
print(mm)
print(ms)

(6, 372028)
(6, 379551)
[[ 0.87500975  0.90780635  0.92082692 ...,  0.24740215  0.78194421
   0.90912815]
 [ 0.52923078  0.75415283  0.8416149  ...,  0.31100479  0.73827392
   0.50788287]
 [ 0.69746529  0.68127254  0.86028262 ...,  0.35032036  0.85354664
   0.29410567]
 [ 0.73969412  0.79942736  0.86677642 ...,  0.49964045  0.63410663
   0.60784865]
 [ 0.          0.          0.         ...,  0.          0.          1.        ]
 [ 0.62100853  0.67671729  0.74663413 ...,  0.33907529  0.68822959
   0.63115047]]
[[ 0.12655612  0.10154384  0.16225399 ...,  0.12070368  0.11114377
   0.10137775]
 [ 0.10727508  0.10103835  0.10189376 ...,  0.10701106  0.10095833
   0.10569277]
 [ 0.11274317  0.10009626  0.10057778 ...,  0.1149717   0.10310947
   0.12588596]
 [ 0.10893174  0.10871207  0.14560457 ...,  0.13492641  0.10616518
   0.18020341]
 [ 0.1         0.1         0.1        ...,  0.1         0.1         0.1       ]
 [ 0.11976167  0.1208131   0.18980543 ...,  0.13622783  0.10435932
   0.10029

In [36]:
# leave one out CV
# TODO: change from mean imputation

# X_np = X_np[:][:33]
# print(X_np.shape)

# o = np.ones(len(train_df))
# o[train_ix] = 0
# o[test_ix] = 0
# unobserved_untested_ix = np.where(o)[0]
# o = np.zeros(len(train_df))
# o[test_ix] = 1
# unobserved_tested_ix = np.where(o)[0]
# perm = np.hstack((train_ix, unobserved_tested_ix, unobserved_untested_ix))

# X_np_perm = X_np[:, perm]

print(X_np.shape)
X_np_new = X_np[:][:33]
X_np_new = np.delete(X_np_new, [14,25,26], 0)
print(X_np_new.shape)

o = np.ones(len(train_df))
o[train_ix] = 0
o[test_ix] = 0
unobserved_untested_ix = np.where(o)[0]
o = np.zeros(len(train_df))
o[test_ix] = 1
unobserved_tested_ix = np.where(o)[0]

perm = np.hstack((train_ix, unobserved_tested_ix, unobserved_untested_ix))
X_np_perm = X_np_new[:, perm]
args_K = range(2, 11)

# these are global variables used in get_avg_r2_cv; don't change
rmse_matrix = np.empty((len(args_K), X_np_perm.shape[0]))
r2_matrix = np.empty((len(args_K), X_np_perm.shape[0]))

def get_avg_r2_cv(K, index):
    folds = range(0, X_np_perm.shape[0])
    r2_arr = []
    rmse_arr = []
    for fold in folds: 
        tf.reset_default_graph()
        sess = tf.InteractiveSession()
        print("K: " + str(K) + ", Fold: " + str(fold))
        X_test = X_np_perm[fold, :]
        X_train = np.delete(X_np_perm, fold, 0)
        print(X_train.shape)
        rmu = np.random.choice(range(X_train.shape[0]), K, replace=False)
        mu_init = X_train[rmu]
        lp, m, s, a = fit_em(X_train, mu_init, 100, EPS, sess)
        print("done fitting em")

        observed = X_test[:len(train_ix)]
        marginal_means, marginal_covs, marginal_alphas = marginal_posterior(observed.reshape(1, len(observed)), m, s, a, sess)

        pred = argmax_exp(marginal_means, marginal_covs, marginal_alphas[0])[:len(unobserved_tested_ix)]
        actual = X_test[len(train_ix):len(train_ix)+len(unobserved_tested_ix)]
        print(len(pred), len(actual))
        rmse = sklearn.metrics.mean_squared_error(actual, pred)
        rmse_arr.append(rmse)
        print('rmse', np.sqrt(rmse)) # rmse of GMM
        r2 = sklearn.metrics.r2_score(actual, pred)
        r2_arr.append(r2)
        print('r2', r2)
        sess.close()
#     return rmse_arr, r2_arr
    rmse_matrix[index, :] = rmse_arr
    r2_matrix[index, :] = r2_arr

# results = Parallel(n_jobs = 2)(delayed(get_avg_r2_cv)(K) for K in args_K)
threads = [None] * len(args_K)
for i in range(len(args_K)):
    threads[i] = threading.Thread(target=get_avg_r2_cv, args=(args_K[i], i))
    threads[i].start()
for i in range(len(args_K)):
    threads[i].join()
    
# i = 0
# for K in args_K:
#     print("K: " + str(K))
#     rmse_arr, r2_arr = get_avg_r2(K)
#     rmse_matrix[i, :] = rmse_arr
#     r2_matrix[i, :] = r2_arr
#     i += 1
    
print(rmse_matrix)
print(r2_matrix)
    

(34, 379551)
(30, 379551)
K: 2, Fold: 0
K: 3, Fold: 0
K: 4, Fold: 0
K: 6, Fold: 0
K: 5, Fold: 0
K: 7, Fold: 0
K: 8, Fold: 0
K: 9, Fold: 0
K: 10, Fold: 0
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)
(29, 379551)


KeyboardInterrupt: 

In [49]:
# save RMSE / R^2
np.save('cv_rmse', rmse_matrix)
np.save('cv_r2', r2_matrix)

array([[ 0.01181451,  0.49466861,  0.01504508,  0.01638574,  0.01518659,
         0.56034061,  0.01619131,  0.01332102,  0.01451038,  0.01450137,
         0.48785416,  0.57519979,  0.66537731,  0.48272767,  0.50345909,
         0.68704757,  0.49897608,  0.62765526,  0.55283239,  0.55880595,
         0.01287547,  0.55201539,  0.01498285,  0.3703144 ,  0.50286533,
         0.56881062,  0.57653486,  0.01450401],
       [ 0.00931847,  0.49466861,  0.00855021,  0.01104168,  0.00829979,
         0.56034061,  0.00910705,  0.00846965,  0.00617491,  0.01058723,
         0.48785416,  0.01933833,  0.01144868,  0.51624171,  0.01938707,
         0.68704757,  0.01679222,  0.62765526,  0.55283239,  0.55880595,
         0.0075879 ,  0.55201539,  0.00841726,  0.3703144 ,  0.0196157 ,
         0.01934769,  0.57653486,  0.01396652],
       [ 0.00840094,  0.49466861,  0.00694016,  0.01137108,  0.00768767,
         0.02823028,  0.00754357,  0.00604806,  0.00552677,  0.00868491,
         0.01546147,  0.0163